## Imports

In [4]:
import tensorflow as tf

In [67]:
import pandas as pd
import os, random, shutil

In [68]:
from sklearn.model_selection import train_test_split

In [3]:
import torch
print("CUDA disponible:", torch.cuda.is_available())
print("Versión PyTorch:", torch.__version__)
print("Versión CUDA usada:", torch.version.cuda)
print("Dispositivo:", torch.cuda.get_device_name(0))

CUDA disponible: True
Versión PyTorch: 2.5.1+cu121
Versión CUDA usada: 12.1
Dispositivo: NVIDIA GeForce RTX 3050 Laptop GPU


In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("robikscube/textocr-text-extraction-from-images-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\andre\.cache\kagglehub\datasets\robikscube\textocr-text-extraction-from-images-dataset\versions\2


## Manejo de archivo csv

In [9]:
print("Carpetas dentro: ", os.listdir(path))

Carpetas dentro:  ['annot.csv', 'annot.parquet', 'img.csv', 'img.parquet', 'TextOCR_0.1_train.json', 'train_val_images']


In [20]:
annot = pd.read_csv(path + '/annot.csv')
annot.head(5)


,Unnamed: 0,id,image_id,bbox,utf8_string,points,area
0,0,a4ea732cd3d5948a_1,a4ea732cd3d5948a,"[525.83, 3.4, 197.64, 33.94]",Performance,"[525.83, 3.4, 723.47, 7.29, 722.76, 36.99, 525...",6707.90
1,1,a4ea732cd3d5948a_2,a4ea732cd3d5948a,"[534.67, 64.68, 91.22, 38.19]",Sport,"[535.73, 64.68, 623.41, 67.51, 625.89, 102.87,...",3483.69
2,2,a4ea732cd3d5948a_3,a4ea732cd3d5948a,"[626.95, 63.62, 96.52, 31.82]",Watch,"[626.95, 63.62, 721.7, 63.62, 723.47, 95.44, 6...",3071.27
3,3,a4ea732cd3d5948a_4,a4ea732cd3d5948a,"[577.4, 141.87, 147.13, 43.1]",...period.,"[580.02, 143.61, 724.53, 141.87, 723.66, 184.9...",6341.30
4,4,a4ea732cd3d5948a_5,a4ea732cd3d5948a,"[391.03, 163.9, 60.82, 38.65]",.,"[395.2, 163.9, 451.85, 191.94, 445.59, 202.55,...",2350.69


In [ ]:
info = pd.read_csv(path + '/img.csv')
info.head(5)

,Unnamed: 0,id,width,height,set,file_name
0,0,a4ea732cd3d5948a,840,1024,train,train/a4ea732cd3d5948a.jpg
1,1,4bf43a7b2a898044,1024,683,train,train/4bf43a7b2a898044.jpg
2,2,1b55b309b0f50d02,1024,683,train,train/1b55b309b0f50d02.jpg
3,3,00c359f294f7dcd9,1024,680,train,train/00c359f294f7dcd9.jpg
4,4,04b5a37f762b0f51,768,1024,train,train/04b5a37f762b0f51.jpg


In [17]:
info = info.rename(columns={'id': 'image_id'})
info.head(5)

,Unnamed: 0,image_id,width,height,set,file_name
0,0,a4ea732cd3d5948a,840,1024,train,train/a4ea732cd3d5948a.jpg
1,1,4bf43a7b2a898044,1024,683,train,train/4bf43a7b2a898044.jpg
2,2,1b55b309b0f50d02,1024,683,train,train/1b55b309b0f50d02.jpg
3,3,00c359f294f7dcd9,1024,680,train,train/00c359f294f7dcd9.jpg
4,4,04b5a37f762b0f51,768,1024,train,train/04b5a37f762b0f51.jpg


In [ ]:
info.drop_duplicates(subset=['image_id'], keep='first', inplace=True)
annot.drop_duplicates(subset=['image_id'], keep='first', inplace=True)
print("Cantidad de anotaciones:", len(annot))
print("Cantidad de imágenes:", len(info))

Cantidad de anotaciones: 1052354
Cantidad de imágenes: 21778


In [26]:
info.drop_duplicates(subset=['image_id'], keep='first', inplace=True)
annot.drop_duplicates(subset=['image_id'], keep='first', inplace=True)
print("Cantidad de anotaciones:", len(annot))
print("Cantidad de imágenes:", len(info))

Cantidad de anotaciones: 21778
Cantidad de imágenes: 21778


In [51]:
img_info = pd.merge(annot, info, how = 'inner', on = 'image_id')
img_info.tail(5)

,Unnamed: 0_x,id,image_id,bbox,utf8_string,points,area,Unnamed: 0_y,width,height,set,file_name
21773,1052200,d13b7ec72bd5eed5_1,d13b7ec72bd5eed5,"[517.02, 355.52, 74.5, 77.5]",FOR,"[517.02, 355.52, 591.02, 355.52, 591.52, 432.5...",5773.75,21773,1024,683,train,train/d13b7ec72bd5eed5.jpg
21774,1052214,0022933cdceee189_1,0022933cdceee189,"[-0.11, 215.6, 43.2, 47.87]",20,"[1.06, 215.6, 43.09, 215.6, 43.09, 259.97, -0....",2067.98,21774,1024,554,train,train/0022933cdceee189.jpg
21775,1052266,6029c75e0325d164_1,6029c75e0325d164,"[486.62, 71.33, 163.65, 66.23]",ORIOLES,"[486.62, 71.65, 650.27, 71.33, 649.95, 137.56,...",10838.54,21775,1024,768,train,train/6029c75e0325d164.jpg
21776,1052335,0ebbecdc46b78d42_1,0ebbecdc46b78d42,"[915.45, 15.84, 67.75, 87.95]",A,"[915.45, 15.84, 982.89, 15.84, 983.2, 103.79, ...",5958.61,21776,1024,681,train,train/0ebbecdc46b78d42.jpg
21777,1052350,a37e1fb026b80a6d_1,a37e1fb026b80a6d,"[331.69, 462.84, 417.31, 201.08]",RÖR,"[331.69, 466.97, 749.0, 462.84, 749.0, 659.79,...",83912.69,21777,1024,683,train,train/a37e1fb026b80a6d.jpg


In [49]:
print("Cantidad de anotaciones e imágenes combinadas:", len(img_info))

Cantidad de anotaciones e imágenes combinadas: 21778


In [52]:
img_info = img_info.drop(columns = ['Unnamed: 0_x', 'id', 'set'])
img_info = img_info.rename(columns={'Unnamed: 0_y': 'img_num'})
img_info['file_name'] = img_info['file_name'].str.replace('train/', '')
img_info.head(5)

,image_id,bbox,utf8_string,points,area,img_num,width,height,file_name
0,a4ea732cd3d5948a,"[525.83, 3.4, 197.64, 33.94]",Performance,"[525.83, 3.4, 723.47, 7.29, 722.76, 36.99, 525...",6707.90,0,840,1024,a4ea732cd3d5948a.jpg
1,4bf43a7b2a898044,"[429.44, 163.75, 27.52, 13.0]",400,"[429.44, 163.75, 456.74, 163.75, 456.96, 176.5...",357.76,1,1024,683,4bf43a7b2a898044.jpg
2,1b55b309b0f50d02,"[564.0, 328.58, 103.5, 41.15]",CAOL,"[564.96, 328.58, 667.5, 338.55, 667.18, 369.73...",4259.02,2,1024,683,1b55b309b0f50d02.jpg
3,00c359f294f7dcd9,"[322.37, 368.64, 118.63, 51.07]",G-ATCO,"[322.37, 391.39, 436.35, 368.64, 441.0, 400.44...",6058.43,3,1024,680,00c359f294f7dcd9.jpg
4,04b5a37f762b0f51,"[217.83, 302.27, 98.68, 45.38]",OUR,"[218.66, 302.27, 316.51, 303.52, 315.68, 347.6...",4478.10,4,768,1024,04b5a37f762b0f51.jpg


In [30]:
img_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21778 entries, 0 to 21777
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   image_id     21778 non-null  object 
 1   bbox         21778 non-null  object 
 2   utf8_string  21774 non-null  object 
 3   points       21778 non-null  object 
 4   area         21778 non-null  float64
 5   img_num      21778 non-null  int64  
 6   width        21778 non-null  int64  
 7   height       21778 non-null  int64  
 8   file_name    21778 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 1.5+ MB


In [31]:
img_info.describe()

,area,img_num,width,height
count,21778.000000,21778.000000,21778.000000,21778.000000
mean,11161.740411,10888.500000,948.640096,817.007439
std,29321.633196,6286.911417,152.755971,163.140660
min,14.560000,0.000000,259.000000,257.000000
25%,832.292500,5444.250000,956.250000,683.000000
50%,2610.220000,10888.500000,1024.000000,768.000000
75%,8802.987500,16332.750000,1024.000000,1024.000000
max,808531.050000,21777.000000,6000.000000,4608.000000


## Lectura de imágenes

In [96]:
imgs_path = path + '/train_val_images/train_images'
all_images = [os.path.join(imgs_path, f) for f in os.listdir(imgs_path)]
random.seed(42)
sample_images = random.sample(all_images, 500)
print("Cantidad de imágenes muestreadas:", len(sample_images))

Cantidad de imágenes muestreadas: 500


In [97]:
sample_path = os.path.join(path, "sample")
if not os.path.exists(sample_path):
    os.makedirs(sample_path)

for img in sample_images:
    shutil.copy(img, sample_path)

In [99]:
sample_imgs_ids = [os.path.basename(f) for f in sample_images]
print("Nombres de archivos muestreados:", sample_imgs_ids[:5])

img_info_sampled = img_info[img_info['file_name'].isin(sample_imgs_ids)]
print("Cantidad de registros en img_info para las imágenes muestreadas:", len(img_info_sampled))

Nombres de archivos muestreados: ['cc63b113d10a3d31.jpg', '0f0ca36db877fc6b.jpg', '014afe1edd4ca5d8.jpg', 'f777d5a22e3b367e.jpg', '412b31ce905437ea.jpg']
Cantidad de registros en img_info para las imágenes muestreadas: 446


In [107]:
for f in sample_imgs_ids:
    if f not in img_info_sampled['file_name'].values:
        # Eliminar imagen de la carpeta
        os.remove(os.path.join(sample_path, f))
        # Eliminar del sample_imgs_ids
        sample_imgs_ids.remove(f)

print("Cantidad final de imágenes muestreadas:", len(sample_imgs_ids))

sample_images = [os.path.join(sample_path, f) for f in os.listdir(sample_path)]
print("Cantidad final de imágenes en la carpeta sample:", len(sample_images))


Cantidad final de imágenes muestreadas: 446
Cantidad final de imágenes en la carpeta sample: 446


In [109]:
train_path = os.path.join(path, "train")
val_path = os.path.join(path, "val")
test_path = os.path.join(path, "test")

for dir in [train_path, val_path, test_path]:
    if not os.path.exists(dir):
        os.makedirs(dir)

train_samp, temp_samp = train_test_split(sample_images, test_size=0.4, random_state=42)
val_samp, test_samp = train_test_split(temp_samp, test_size=0.5, random_state=42)

for img in train_samp:
    shutil.copy(img, train_path)

for img in val_samp:
    shutil.copy(img, val_path)

for img in test_samp:
    shutil.copy(img, test_path)
    
print("Cantidad de imágenes en train:", len(os.listdir(train_path)))
print("Cantidad de imágenes en val:", len(os.listdir(val_path)))
print("Cantidad de imágenes en test:", len(os.listdir(test_path)))

Cantidad de imágenes en train: 267
Cantidad de imágenes en val: 89
Cantidad de imágenes en test: 90


In [112]:
# Obtener los IDs de las imágenes en cada conjunto
samp_imgs_train_ids = [os.path.basename(f) for f in train_samp]
samp_imgs_val_ids = [os.path.basename(f) for f in val_samp]
samp_imgs_test_ids = [os.path.basename(f) for f in test_samp]

img_info_train = img_info_sampled[img_info_sampled['file_name'].isin(samp_imgs_train_ids)]
img_info_val = img_info_sampled[img_info_sampled['file_name'].isin(samp_imgs_val_ids)]
img_info_test = img_info_sampled[img_info_sampled['file_name'].isin(samp_imgs_test_ids)]

print("Cantidad de registros en img_info para train:", len(img_info_train))
print("Cantidad de registros en img_info para val:", len(img_info_val))
print("Cantidad de registros en img_info para test:", len(img_info_test))

img_info_train.head(5)

Cantidad de registros en img_info para train: 267
Cantidad de registros en img_info para val: 89
Cantidad de registros en img_info para test: 90


,image_id,bbox,utf8_string,points,area,img_num,width,height,file_name
9,a183473b9b4bc8f6,"[166.08, 0.7, 32.97, 326.88]",LIDER/MARCELLIONO,"[199.05, 0.7, 194.75, 326.86, 166.08, 327.58, ...",10777.23,9,357,1024,a183473b9b4bc8f6.jpg
151,42113d4ea98235b5,"[570.17, 204.36, 90.96, 30.32]",McGINLEY,"[572.88, 204.36, 661.13, 213.72, 657.18, 234.6...",2757.91,151,1024,683,42113d4ea98235b5.jpg
320,2efeea2e77a59e8f,"[210.27, 127.41, 42.01, 49.73]",9,"[210.27, 128.27, 248.85, 127.41, 252.28, 175.4...",2089.16,320,480,1024,2efeea2e77a59e8f.jpg
362,03031fa84a2806a3,"[594.47, 89.11, 105.86, 130.54]",L,"[594.47, 98.67, 659.34, 89.11, 700.33, 203.73,...",13818.96,362,891,1024,03031fa84a2806a3.jpg
408,ad7003f84cd50392,"[207.46, 305.13, 29.54, 13.39]",New,"[207.46, 305.13, 236.6, 305.13, 237.0, 317.73,...",395.54,408,1024,768,ad7003f84cd50392.jpg


In [113]:
IMG_SIZE = (512, 512)

def create_dataset(df, subset_name):
    """
    Crea un tf.data.Dataset para un subconjunto (train, val o test)
    a partir del DataFrame correspondiente y su carpeta de imágenes.
    """
    images_path = os.path.join(path, 'train_val_images', f'{subset_name}')

    # Extraer columnas necesarias
    file_names = df['file_name'].values
    texts = df['utf8_string'].values
    bbox = df['bbox'].values
    points = df['points'].values

    # Crear dataset base
    dataset = tf.data.Dataset.from_tensor_slices((file_names, texts, bbox, points))

    # Definir función de carga
    def load_image_and_text(file_name, text, bbox, points):
        image_path = tf.strings.join([images_path, '/', file_name])
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, IMG_SIZE)
        image = image / 255.0
        return image, text, bbox, points

    # Mapear
    dataset = dataset.map(load_image_and_text, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

In [ ]:
# Crear datasets para cada conjunto
train_dataset = create_dataset(img_info_train, "train")
val_dataset = create_dataset(img_info_val, "val")
test_dataset = create_dataset(img_info_test, "test")

print("Train samples:", len(img_info_train))
print("Val samples:", len(img_info_val))
print("Test samples:", len(img_info_test))

Train samples: 267
Val samples: 89
Test samples: 90


<_ParallelMapDataset element_spec=(TensorSpec(shape=(512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

## Modelo